# This goal of this notebook is to create a dataset of US stocks and associated key financial ratios. 
- Steps include:
    - Pull stock tickers from an API for the financial data compiler service Financial Modeling Prep (https://site.financialmodelingprep.com/developer/docs/)
    - Filter to applicable US stocks
    - Join with pricing data and calculate annual return
    - Pull financial ratios and sector data
    - Clean data and export to csv

#
## Note: This notebook was created for educational purposes and to improve data wrangling skills. It should not be used in for financial decisions in any way
#
- Author: Will Merline

In [1]:
from urllib.request import urlopen
import certifi
import json

import numpy as np
import pandas as pd

## Pull stock tickers

In [2]:
# Get stock list
stock_list = ("https://financialmodelingprep.com/api/v3/available-traded/list?apikey=4f3f56682d5bbab7a42251bad1821ade")
print(stock_list)

https://financialmodelingprep.com/api/v3/available-traded/list?apikey=4f3f56682d5bbab7a42251bad1821ade


In [3]:
# Create dataframe
df = pd.read_json(stock_list)

In [4]:
df

,symbol,name,price,exchange,exchangeShortName,type
0,ADN.AX,Andromeda Metals Limited,0.021,Australian Securities Exchange,ASX,stock
1,RTC.DE,RealTech AG,1.060,Frankfurt Stock Exchange,XETRA,stock
2,FUPPF,Fuchs Petrolub SE,41.460,Other OTC,PNK,stock
3,ICSIX,Dynamic U.S. Opportunity Fund - Class I,15.690,NASDAQ,NASDAQ,trust
4,BYU,"BAIYU Holdings, Inc.",1.600,NASDAQ Global Market,NASDAQ,stock
...,...,...,...,...,...,...
54977,ALL,The Allstate Corporation,139.600,New York Stock Exchange,NYSE,stock
54978,MYPS,"PLAYSTUDIOS, Inc.",2.670,NASDAQ Global Market,NASDAQ,stock
54979,600615.SS,"Shanghai Fenghwa Group Co., Ltd",12.700,Shanghai,SHH,stock
54980,102110.KS,Mirae Asset TIGER 200 ETF,34420.000,KSE,KSC,etf


In [4]:
# Check stock exchanges in dataset:
df['exchangeShortName'].unique()

array(['ASX', 'XETRA', 'PNK', 'NASDAQ', 'SAO', 'JPX', 'JKT', 'STO', 'LSE',
       'SET', 'SHZ', 'TSX', 'SHH', 'NSE', 'WSE', 'ATH', 'AMEX', 'NYSE',
       'HKSE', 'SAU', 'KSC', 'TLV', 'JNB', 'ASE', 'KOE', 'OTC',
       'EURONEXT', 'BUE', 'HEL', 'MIL', 'VIE', 'TAI', 'NEO', 'ETF', 'TWO',
       'IST', 'CPH', 'CNQ', 'NZE', 'OSL', 'SIX', 'BUD', 'SES', 'MEX',
       'KLS', 'KUW', 'CBOE', 'SGO', 'BRU', 'DOH', 'BME', 'AMS', 'IOB',
       'ICE', 'DXE', 'RIS', 'STU', 'AQS', 'BSE', 'CAI', 'PRA', 'ISE',
       'DFM', 'HAM', None], dtype=object)

In [5]:
df['exchangeShortName'].value_counts()

NASDAQ    8566
PNK       5730
LSE       5110
NYSE      3079
JPX       2603
          ... 
KUW          3
AQS          2
RIS          1
ISE          1
HAM          1
Name: exchangeShortName, Length: 64, dtype: int64

In [6]:
# Get primary US stock exchanges: NASDAQ and New York Stock Exchange (NYSE):
US_exchanges = ['NASDAQ', 'NYSE']

In [7]:
print("NASDAQ values:", (df['exchangeShortName']=='NASDAQ').sum())
print("NYSE values:", (df['exchangeShortName']=='NYSE').sum())
print("Total:", (df['exchangeShortName']=='NASDAQ').sum() + (df['exchangeShortName']=='NYSE').sum())

NASDAQ values: 8566
NYSE values: 3079
Total: 11645


In [8]:
# Filter stock list to US exchanges:
df_US_stocks = df.loc[df['exchangeShortName'].isin(US_exchanges)]
df_US_stocks

,symbol,name,price,exchange,exchangeShortName,type
3,ICSIX,Dynamic U.S. Opportunity Fund - Class I,15.6900,NASDAQ,NASDAQ,trust
4,BYU,"BAIYU Holdings, Inc.",1.6000,NASDAQ Global Market,NASDAQ,stock
8,MBEYX,AMG GW&K Global Allocation Fund,11.7300,NASDAQ,NASDAQ,trust
10,FAASX,Fidelity Asset Manager 70% Fund,25.7900,NASDAQ,NASDAQ,trust
22,FFRHX,Fidelity Floating Rate High Income,9.2700,NASDAQ,NASDAQ,trust
...,...,...,...,...,...,...
54980,UFI,"Unifi, Inc.",6.2800,New York Stock Exchange,NYSE,stock
54985,GMRE-PA,Global Medical REIT Inc.,25.3201,New York Stock Exchange,NYSE,trust
54986,FHLT,Future Health ESG Corp.,10.5800,NASDAQ Capital Market,NASDAQ,stock
54989,ALL,The Allstate Corporation,139.4550,New York Stock Exchange,NYSE,stock


In [9]:
# Check values after filter:

print("NASDAQ values:", (df_US_stocks['exchangeShortName']=='NASDAQ').sum())
print("NYSE values:", (df_US_stocks['exchangeShortName']=='NYSE').sum())
print("Total:", (df_US_stocks['exchangeShortName']=='NASDAQ').sum() + (df_US_stocks['exchangeShortName']=='NYSE').sum())

NASDAQ values: 8566
NYSE values: 3079
Total: 11645


In [10]:
# Check tickers by product type:

df_US_stocks
df_US_stocks['type'].value_counts()

stock    6799
trust    4254
etf       591
fund        1
Name: type, dtype: int64

In [11]:
# Filter out trusts and etfs:
df_US_stocks2 = df_US_stocks.loc[df_US_stocks['type'] == 'stock']

In [12]:
# Check value  counts after removing trusts and etfs:
df_US_stocks2['type'].value_counts()

stock    6799
Name: type, dtype: int64

In [13]:
df_US_stocks2

,symbol,name,price,exchange,exchangeShortName,type
4,BYU,"BAIYU Holdings, Inc.",1.600,NASDAQ Global Market,NASDAQ,stock
23,BCS,Barclays PLC,7.525,New York Stock Exchange,NYSE,stock
26,CBRE,"CBRE Group, Inc.",91.540,New York Stock Exchange,NYSE,stock
50,TWLO,Twilio Inc.,75.900,New York Stock Exchange,NYSE,stock
59,WBA,"Walgreens Boots Alliance, Inc.",25.435,NASDAQ Global Select,NASDAQ,stock
...,...,...,...,...,...,...
54979,INN-PE,"Summit Hotel Properties, Inc.",19.600,New York Stock Exchange,NYSE,stock
54980,UFI,"Unifi, Inc.",6.280,New York Stock Exchange,NYSE,stock
54986,FHLT,Future Health ESG Corp.,10.580,NASDAQ Capital Market,NASDAQ,stock
54989,ALL,The Allstate Corporation,139.455,New York Stock Exchange,NYSE,stock


In [14]:
# Export to CSV:
df_US_stocks2.to_csv('US_Stock_List.csv')

In [15]:
# Get list of stock tickers:
tickers_list = df_US_stocks2['symbol'].tolist()
tickers_list

['BYU',
 'BCS',
 'CBRE',
 'TWLO',
 'WBA',
 'TWO-PC',
 'ARVN',
 'CSAN',
 'NAPA',
 'GTN',
 'EXPE',
 'EVH',
 'CRGX',
 'EBR-B',
 'SPGC',
 'GOCO',
 'RELY',
 'BHG',
 'TLS',
 'DUK',
 'ANVS',
 'LYRA',
 'NICE',
 'SMFG',
 'BTE',
 'ACTG',
 'ABSI',
 'SITC',
 'PRTC',
 'PRPH',
 'AMWL',
 'MDAI',
 'KGS',
 'TIRX',
 'CHSN',
 'XPO',
 'ESCA',
 'POAI',
 'KDP',
 'EVGRU',
 'LFT-PA',
 'CBZ',
 'DRH-PA',
 'BHR',
 'MARK',
 'GLDD',
 'ANSCU',
 'EFOI',
 'WNNR-UN',
 'BWAQ',
 'FTFT',
 'PSNY',
 'AREC',
 'WTRG',
 'OSUR',
 'AQN',
 'ASBA',
 'SB-PD',
 'DUETU',
 'KACL',
 'TRIS',
 'GRIN',
 'MCFT',
 'CTMX',
 'FWRG',
 'BIGC',
 'HLF',
 'AAN',
 'VLRS',
 'CAR',
 'INSP',
 'OXLCO',
 'MRSN',
 'PETZ',
 'IIIV',
 'JOE',
 'MUFG',
 'PTON',
 'TRON',
 'CRGY',
 'TDW',
 'AG',
 'MBWM',
 'FATH',
 'RILYP',
 'CMRE-PC',
 'EVRG',
 'RITM-PB',
 'AIH',
 'BRO',
 'PCTTW',
 'DBI',
 'FA',
 'PRIF-PJ',
 'HQL',
 'XGN',
 'AITRU',
 'BE',
 'VMAR',
 'WTTR',
 'CIB',
 'DZSI',
 'DBRG-PH',
 'MITT-PB',
 'XYL',
 'ATCO-PH',
 'DTC',
 'SLMBP',
 'VMCAW',
 'EDBL',
 'IPW'

In [16]:
# Convert tickers list to dataframe: 
df_tickers = pd.DataFrame(tickers_list, columns=['symbol'])

df_tickers

,symbol
0,BYU
1,BCS
2,CBRE
3,TWLO
4,WBA
...,...
6794,INN-PE
6795,UFI
6796,FHLT
6797,ALL


In [17]:
# Saving the Tickers dataframe to CSV file:
df_tickers.to_csv('Stock_Tickers_v1.csv')
print("Stock tickers exported to working directory")

Stock tickers exported to working directory


Note: Pricing data was downloaded into CSV files and stored locally. Pricing data was sourced from Financial Modeling Prep (https://site.financialmodelingprep.com/developer/docs/) using a free trial

In [30]:
# Read annual pricing csv back to dataframes:
df_2016 = pd.read_csv('Data/batch_prices_eod_2016.csv')
df_2017 = pd.read_csv('Data/batch_prices_eod_2017.csv')
df_2018 = pd.read_csv('Data/batch_prices_eod_2018.csv')
df_2019 = pd.read_csv('Data/batch_prices_eod_2019.csv')
df_2020 = pd.read_csv('Data/batch_prices_eod_2020.csv')
df_2021 = pd.read_csv('Data/batch_prices_eod_2021.csv')
df_2022 = pd.read_csv('Data/batch_prices_eod_2022.csv')


In [31]:
# Combine pricing dfs
df_prices = pd.concat([df_2016, df_2017, df_2018, df_2019, df_2020, df_2021, df_2022], axis=0, ignore_index=True)
df_prices

,symbol,date,open,low,high,close,adjClose,volume
0,SNDE,12/30/2016,159.5000,159.1500,159.500,159.200,159.200000,100
1,AMOT,12/30/2016,14.1587,14.1133,14.580,14.260,13.990335,10908
2,8271.HK,12/30/2016,0.3100,0.3000,0.350,0.330,0.252353,564000
3,BOL.PA,12/30/2016,3.3600,3.3250,3.366,3.350,3.059407,1051663
4,2369.TW,12/30/2016,11.7500,11.7500,11.900,11.850,9.891171,1003000
...,...,...,...,...,...,...,...,...
348191,0I9F.L,12/30/2022,101.1500,100.1400,101.150,100.140,100.140000,3
348192,0IC7.L,12/30/2022,247.0000,245.0450,247.330,245.045,245.045000,26
348193,0IC8.L,12/30/2022,140.8100,140.8100,140.810,140.810,140.810000,704
348194,0IC9.L,12/30/2022,61.7000,60.7100,61.700,60.710,60.710000,10


In [32]:
# Inner Join df_prices with stock tickers:
df_prices2 = pd.merge(df_prices, df_tickers, how='inner', on = 'symbol')

In [33]:
# View dataframe:
df_prices2

,symbol,date,open,low,high,close,adjClose,volume
0,RELL,12/30/2016,6.26000,6.15000,6.300,6.3000,5.098714,21365
1,RELL,12/29/2017,6.56890,6.56890,6.750,6.7400,5.674603,5555
2,RELL,12/31/2018,9.00000,8.44780,9.000,8.6900,7.528636,140556
3,RELL,12/31/2019,5.67000,5.61000,5.740,5.6300,5.076638,19503
4,RELL,12/31/2020,4.74260,4.68000,4.753,4.7100,4.485001,6997
...,...,...,...,...,...,...,...,...
32936,VCXB,12/30/2022,10.17000,9.77000,10.180,9.7700,9.770000,50082
32937,VCXB-UN,12/30/2022,10.98500,10.45000,11.010,11.0100,11.010000,4534
32938,VCXB-WT,12/30/2022,0.07525,0.07525,0.080,0.0800,0.080000,1060
32939,BNRG,12/30/2022,1.48000,1.37210,1.480,1.3721,1.372000,449


In [34]:
# Check for NAs:
df_prices2.isna().sum()/len(df_prices2)

symbol      0.0
date        0.0
open        0.0
low         0.0
high        0.0
close       0.0
adjClose    0.0
volume      0.0
dtype: float64

In [35]:
# Saving the pricing dataframe to CSV file:
df_prices2.to_csv('Prices_v1.csv')
print("Stock tickers exported to working directory")

Stock tickers exported to working directory


## Explortatory Data Analysis

In [36]:
# Check unique values:
df_prices2['symbol'].value_counts()


RELL       7
EDIT       7
VTGN       7
SLRX       7
PALI       7
          ..
SCCF       1
BCAN       1
BBUC       1
BBAI-WT    1
BOCN       1
Name: symbol, Length: 6425, dtype: int64

In [37]:
# Check symbols with value counts equal to 1 (only 1 year of data):

print('Count of symbols = 1:', (df_prices2['symbol'].value_counts() == 1).sum())
print('Count of symbols > 1:', (df_prices2['symbol'].value_counts() > 1).sum())

Count of symbols = 1: 472
Count of symbols > 1: 5953


In [38]:
# Remove symbols where value count = 1:
df_prices3 = df_prices2.copy()

df_prices3 = df_prices3[df_prices3.groupby('symbol').symbol.transform('count')>1]
df_prices3


,symbol,date,open,low,high,close,adjClose,volume
0,RELL,12/30/2016,6.2600,6.1500,6.3000,6.300,5.098714,21365
1,RELL,12/29/2017,6.5689,6.5689,6.7500,6.740,5.674603,5555
2,RELL,12/31/2018,9.0000,8.4478,9.0000,8.690,7.528636,140556
3,RELL,12/31/2019,5.6700,5.6100,5.7400,5.630,5.076638,19503
4,RELL,12/31/2020,4.7426,4.6800,4.7530,4.710,4.485001,6997
...,...,...,...,...,...,...,...,...
32471,BNIX,12/30/2022,10.1700,10.1700,10.1701,10.170,10.170000,26025
32472,BNIXW,12/31/2021,0.4700,0.4700,0.4700,0.470,0.470000,500
32473,BNIXW,12/30/2022,0.0250,0.0250,0.0250,0.025,0.025000,0
32474,BODY,12/31/2021,2.3800,2.3400,2.5000,2.370,2.370000,1617034


In [39]:
# Check symbols with value counts after removing counts = 1:
print('Count of symbols = 1:', (df_prices3['symbol'].value_counts() == 1).sum())
print('Count of symbols > 1:', (df_prices3['symbol'].value_counts() > 1).sum())

Count of symbols = 1: 0
Count of symbols > 1: 5953


In [18]:
#list_test = ['AAPL', 'META', 'error']
#len(list_test)

3

### Apply year and annual return

In [40]:
# Keep important columns: symbol, date, open, high, low, close, adjClose
cols = ['symbol', 'date', 'close', 'adjClose']
df_prices3 = df_prices3[cols]
df_prices3

,symbol,date,close,adjClose
0,RELL,12/30/2016,6.300,5.098714
1,RELL,12/29/2017,6.740,5.674603
2,RELL,12/31/2018,8.690,7.528636
3,RELL,12/31/2019,5.630,5.076638
4,RELL,12/31/2020,4.710,4.485001
...,...,...,...,...
32471,BNIX,12/30/2022,10.170,10.170000
32472,BNIXW,12/31/2021,0.470,0.470000
32473,BNIXW,12/30/2022,0.025,0.025000
32474,BODY,12/31/2021,2.370,2.370000


In [27]:
# Last trading day of each year: YYYY-MM-DD
    # 2016: 2016-12-30
    # 2017: 2017-12-29
    # 2018: 2018-12-31
    # 2019: 2019-12-31
    # 2020: 2020-12-31
    # 2021: 2021-12-31
    # 2022: 2022-12-30

#Dates = ['2022-12-30', '2021-12-31', '2020-12-31', '2019-12-31','2018-12-31']
#Dates

['2022-12-30', '2021-12-31', '2020-12-31', '2019-12-31', '2018-12-31']

In [41]:
# Add a year column and sort in descending order:

df_prices4 = df_prices3.copy()
df_prices4['Year'] = pd.DatetimeIndex(df_prices4['date']).year
df_prices4.sort_values(by=['symbol', 'Year'], ignore_index=True, inplace=True, ascending=False)


df_prices4

,symbol,date,close,adjClose,Year
0,ZYXI,12/30/2022,13.910000,13.910000,2022
1,ZYXI,12/31/2021,9.063600,8.964234,2021
2,ZYXI,12/31/2020,12.236400,12.102167,2020
3,ZYXI,12/31/2019,7.154500,7.076080,2019
4,ZYXI,12/31/2018,2.644294,2.643415,2018
...,...,...,...,...,...
32464,A,12/31/2020,118.490000,116.890472,2020
32465,A,12/31/2019,85.310000,83.618065,2019
32466,A,12/31/2018,67.460000,65.562431,2018
32467,A,12/29/2017,66.970000,64.488472,2017


In [42]:
df_prices4['PY_adjClose'] = 0
df_prices4

,symbol,date,close,adjClose,Year,PY_adjClose
0,ZYXI,12/30/2022,13.910000,13.910000,2022,0
1,ZYXI,12/31/2021,9.063600,8.964234,2021,0
2,ZYXI,12/31/2020,12.236400,12.102167,2020,0
3,ZYXI,12/31/2019,7.154500,7.076080,2019,0
4,ZYXI,12/31/2018,2.644294,2.643415,2018,0
...,...,...,...,...,...,...
32464,A,12/31/2020,118.490000,116.890472,2020,0
32465,A,12/31/2019,85.310000,83.618065,2019,0
32466,A,12/31/2018,67.460000,65.562431,2018,0
32467,A,12/29/2017,66.970000,64.488472,2017,0


In [43]:
# Create next_row_symbol column for linking current year close with Prior year close
df_prices4['Offset_symbol'] =  0
df_prices4

,symbol,date,close,adjClose,Year,PY_adjClose,Offset_symbol
0,ZYXI,12/30/2022,13.910000,13.910000,2022,0,0
1,ZYXI,12/31/2021,9.063600,8.964234,2021,0,0
2,ZYXI,12/31/2020,12.236400,12.102167,2020,0,0
3,ZYXI,12/31/2019,7.154500,7.076080,2019,0,0
4,ZYXI,12/31/2018,2.644294,2.643415,2018,0,0
...,...,...,...,...,...,...,...
32464,A,12/31/2020,118.490000,116.890472,2020,0,0
32465,A,12/31/2019,85.310000,83.618065,2019,0,0
32466,A,12/31/2018,67.460000,65.562431,2018,0,0
32467,A,12/29/2017,66.970000,64.488472,2017,0,0


In [44]:
# Fill offset symbol column with symbol from one row below
j = 1
for i in df_prices4['Offset_symbol']:
    try:
        df_prices4['Offset_symbol'].iloc[j-1] = df_prices4['symbol'].iloc[j]
        j = j+1
    except:
        print('\n' 'Last row, no value to fill!')

df_prices4

C:\Users\willm\AppData\Local\Temp\ipykernel_10212\2888858565.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prices4['Offset_symbol'].iloc[j-1] = df_prices4['symbol'].iloc[j]



Last row, no value to fill!


,symbol,date,close,adjClose,Year,PY_adjClose,Offset_symbol
0,ZYXI,12/30/2022,13.910000,13.910000,2022,0,ZYXI
1,ZYXI,12/31/2021,9.063600,8.964234,2021,0,ZYXI
2,ZYXI,12/31/2020,12.236400,12.102167,2020,0,ZYXI
3,ZYXI,12/31/2019,7.154500,7.076080,2019,0,ZYXI
4,ZYXI,12/31/2018,2.644294,2.643415,2018,0,ZYXI
...,...,...,...,...,...,...,...
32464,A,12/31/2020,118.490000,116.890472,2020,0,A
32465,A,12/31/2019,85.310000,83.618065,2019,0,A
32466,A,12/31/2018,67.460000,65.562431,2018,0,A
32467,A,12/29/2017,66.970000,64.488472,2017,0,A


In [45]:
# fill prior year adjusted close column
    ## NA rows to be deleted
j = 0
for i in df_prices4['PY_adjClose']:
    if df_prices4['Offset_symbol'].iloc[j] == df_prices4['symbol'].iloc[j]:
        df_prices4['PY_adjClose'].iloc[j] = df_prices4['adjClose'].iloc[j+1]

    else:
        df_prices4['PY_adjClose'].iloc[j] = 0
        
    j = j+1

df_prices4

C:\Users\willm\AppData\Local\Temp\ipykernel_10212\1504632125.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_prices4['PY_adjClose'].iloc[j] = df_prices4['adjClose'].iloc[j+1]


,symbol,date,close,adjClose,Year,PY_adjClose,Offset_symbol
0,ZYXI,12/30/2022,13.910000,13.910000,2022,8.964234,ZYXI
1,ZYXI,12/31/2021,9.063600,8.964234,2021,12.102167,ZYXI
2,ZYXI,12/31/2020,12.236400,12.102167,2020,7.076080,ZYXI
3,ZYXI,12/31/2019,7.154500,7.076080,2019,2.643415,ZYXI
4,ZYXI,12/31/2018,2.644294,2.643415,2018,2.783678,ZYXI
...,...,...,...,...,...,...,...
32464,A,12/31/2020,118.490000,116.890472,2020,83.618065,A
32465,A,12/31/2019,85.310000,83.618065,2019,65.562431,A
32466,A,12/31/2018,67.460000,65.562431,2018,64.488472,A
32467,A,12/29/2017,66.970000,64.488472,2017,43.479897,A


In [46]:
# add a key/index column consisting of symbol + year
df_prices4['Key'] = df_prices4['symbol'].astype(str) + "-" + df_prices4['Year'].astype(str)
df_prices4

,symbol,date,close,adjClose,Year,PY_adjClose,Offset_symbol,Key
0,ZYXI,12/30/2022,13.910000,13.910000,2022,8.964234,ZYXI,ZYXI-2022
1,ZYXI,12/31/2021,9.063600,8.964234,2021,12.102167,ZYXI,ZYXI-2021
2,ZYXI,12/31/2020,12.236400,12.102167,2020,7.076080,ZYXI,ZYXI-2020
3,ZYXI,12/31/2019,7.154500,7.076080,2019,2.643415,ZYXI,ZYXI-2019
4,ZYXI,12/31/2018,2.644294,2.643415,2018,2.783678,ZYXI,ZYXI-2018
...,...,...,...,...,...,...,...,...
32464,A,12/31/2020,118.490000,116.890472,2020,83.618065,A,A-2020
32465,A,12/31/2019,85.310000,83.618065,2019,65.562431,A,A-2019
32466,A,12/31/2018,67.460000,65.562431,2018,64.488472,A,A-2018
32467,A,12/29/2017,66.970000,64.488472,2017,43.479897,A,A-2017


In [47]:
# Calculate annual return: (close / PY_adjClose) - 1
    ## Adjusted close is used as it incorporates stock splits, dividends, and new offerings
    ## Adjusted close is industry standard for calculating historical returns
df_prices4['Return'] = (df_prices4['adjClose'].astype(float) / df_prices4['PY_adjClose'].astype(float)) - 1
#df_prices4 = df_prices4.drop(columns=['Return%'])
df_prices4

,symbol,date,close,adjClose,Year,PY_adjClose,Offset_symbol,Key,Return
0,ZYXI,12/30/2022,13.910000,13.910000,2022,8.964234,ZYXI,ZYXI-2022,0.551722
1,ZYXI,12/31/2021,9.063600,8.964234,2021,12.102167,ZYXI,ZYXI-2021,-0.259287
2,ZYXI,12/31/2020,12.236400,12.102167,2020,7.076080,ZYXI,ZYXI-2020,0.710293
3,ZYXI,12/31/2019,7.154500,7.076080,2019,2.643415,ZYXI,ZYXI-2019,1.676871
4,ZYXI,12/31/2018,2.644294,2.643415,2018,2.783678,ZYXI,ZYXI-2018,-0.050388
...,...,...,...,...,...,...,...,...,...
32464,A,12/31/2020,118.490000,116.890472,2020,83.618065,A,A-2020,0.397909
32465,A,12/31/2019,85.310000,83.618065,2019,65.562431,A,A-2019,0.275396
32466,A,12/31/2018,67.460000,65.562431,2018,64.488472,A,A-2018,0.016654
32467,A,12/29/2017,66.970000,64.488472,2017,43.479897,A,A-2017,0.483179


In [48]:
df_prices_clean = df_prices4.copy()
df_prices_clean


,symbol,date,close,adjClose,Year,PY_adjClose,Offset_symbol,Key,Return
0,ZYXI,12/30/2022,13.910000,13.910000,2022,8.964234,ZYXI,ZYXI-2022,0.551722
1,ZYXI,12/31/2021,9.063600,8.964234,2021,12.102167,ZYXI,ZYXI-2021,-0.259287
2,ZYXI,12/31/2020,12.236400,12.102167,2020,7.076080,ZYXI,ZYXI-2020,0.710293
3,ZYXI,12/31/2019,7.154500,7.076080,2019,2.643415,ZYXI,ZYXI-2019,1.676871
4,ZYXI,12/31/2018,2.644294,2.643415,2018,2.783678,ZYXI,ZYXI-2018,-0.050388
...,...,...,...,...,...,...,...,...,...
32464,A,12/31/2020,118.490000,116.890472,2020,83.618065,A,A-2020,0.397909
32465,A,12/31/2019,85.310000,83.618065,2019,65.562431,A,A-2019,0.275396
32466,A,12/31/2018,67.460000,65.562431,2018,64.488472,A,A-2018,0.016654
32467,A,12/29/2017,66.970000,64.488472,2017,43.479897,A,A-2017,0.483179


In [49]:
# Export clean prices to csv
df_prices_clean.to_csv('Prices_Clean.csv')

In [50]:
tickers_df = df_prices_clean['symbol']
tickers_df = tickers_df.drop_duplicates()
tickers_list = tickers_df.values.tolist()
tickers_list

['ZYXI',
 'ZYME',
 'ZXZZT',
 'ZXYZ-A',
 'ZWS',
 'ZVIA',
 'ZUO',
 'ZUMZ',
 'ZTST',
 'ZTS',
 'ZTO',
 'ZTEK',
 'ZS',
 'ZNTL',
 'ZM',
 'ZLAB',
 'ZKIN',
 'ZJZZT',
 'ZIVOW',
 'ZIVO',
 'ZIP',
 'ZIONP',
 'ZIONO',
 'ZIONL',
 'ZION',
 'ZIM',
 'ZI',
 'ZH',
 'ZGN',
 'ZG',
 'ZFOXW',
 'ZFOX',
 'ZEV',
 'ZEUS',
 'ZETA',
 'ZEPP',
 'ZENV',
 'ZD',
 'ZCMD',
 'ZBRA',
 'ZBH',
 'Z',
 'YY',
 'YUMC',
 'YUM',
 'YTRA',
 'YTEN',
 'YSG',
 'YS',
 'YRD',
 'YQ',
 'YPF',
 'YOU',
 'YORW',
 'YMM',
 'YMAB',
 'YJ',
 'YI',
 'YGMZ',
 'YEXT',
 'YETI',
 'YELP',
 'YALA',
 'XYL',
 'XYF',
 'XXII',
 'XWEL',
 'XTLB',
 'XRX',
 'XRTX',
 'XRAY',
 'XPRO',
 'XPOF',
 'XPO',
 'XPEV',
 'XPER',
 'XPEL',
 'XPDBU',
 'XP',
 'XOSWW',
 'XOS',
 'XOMAP',
 'XOMAO',
 'XOMA',
 'XOM',
 'XNET',
 'XNCR',
 'XMTR',
 'XLO',
 'XIN',
 'XHR',
 'XGN',
 'XFOR',
 'XFINW',
 'XFINU',
 'XFIN',
 'XERS',
 'XENE',
 'XELB',
 'XELA',
 'XEL',
 'XCUR',
 'XBIT',
 'XBIO',
 'XAIR',
 'X',
 'WYNN',
 'WY',
 'WWW',
 'WWD',
 'WW',
 'WVVIP',
 'WVVI',
 'WVE',
 'WULF',
 'WU',
 'WTW

## Pull financial ratio data
- Note: this data can be pulled from FMP ("https://financialmodelingprep.com/api/v3/key-metrics/") with the required subscription or can be manually downloaded into CSV files

In [ ]:
## Get financial ratio data from stocks on tickers_list ##

base_ratios = ("https://financialmodelingprep.com/api/v3/key-metrics/")
end_ratios = ("?limit=5&apikey=4f3f56682d5bbab7a42251bad1821ade")       

json_list_ratios = []
for i in tickers_list:
    url_ratios = base_ratios + i + end_ratios
    #print(get_jsonparsed_data(url))
    #print(url)
    json_ratios = pd.read_json(url_ratios)
    json_list_ratios.append(json_ratios)
   

json_list_ratios


In [143]:
# Convert pricing JSON files into dataframe 
df_ratios = pd.concat(json_list_ratios, axis=0, ignore_index=True)
df_ratios

,symbol,date,period,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,bookValuePerShare,tangibleBookValuePerShare,...,averagePayables,averageInventory,daysSalesOutstanding,daysPayablesOutstanding,daysOfInventoryOnHand,receivablesTurnover,payablesTurnover,inventoryTurnover,roe,capexPerShare
0,REGN,2022-12-31,FY,113.659197,40.507937,0.000000,34.586368,72.290383,211.615313,203.067227,...,576600000.0,1.915700e+09,159.779141,137.822353,439.782428,2.284403,2.648337,0.829956,0.191423,34.586368
1,REGN,2021-12-31,FY,149.365242,75.049257,65.811338,60.682156,52.924721,174.431227,174.431227,...,519750000.0,1.933950e+09,137.093307,84.455385,292.194667,2.662420,4.321809,1.249167,0.430251,-5.129182
2,REGN,2020-12-31,FY,78.969331,32.650558,24.331784,18.619888,33.336431,102.465613,102.465613,...,446800000.0,1.666050e+09,176.750362,154.975891,624.662023,2.065059,2.355205,0.584316,0.318649,-5.711896
3,REGN,2019-12-31,FY,72.009158,19.375458,22.252747,18.318681,29.434982,101.554029,101.554029,...,318150000.0,1.283350e+09,124.037058,195.099080,660.518410,2.942669,1.870844,0.552596,0.190790,-3.934066
4,REGN,2018-12-31,FY,62.194625,22.654310,20.343837,16.793327,26.041705,81.161260,81.161260,...,198191500.0,9.386690e+08,122.007510,183.466943,967.952085,2.991619,1.989459,0.377085,0.279127,-3.550510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28871,BODY,2022-12-31,FY,2.251134,-0.631541,-0.153414,-0.239573,0.260468,0.687966,0.254227,...,33159500.0,9.339500e+07,0.456646,20.296256,61.160291,799.306005,17.983612,5.967925,-0.917983,-0.086159
28872,BODY,2021-12-31,FY,2.869802,-0.750203,-0.707063,-0.962990,0.341803,1.265936,0.702474,...,38680000.0,9.904200e+07,0.511793,53.727948,147.405085,713.179592,6.793485,2.476170,-0.592607,-0.255927
28873,BODY,2020-12-31,FY,2.836746,-0.070401,0.201789,0.077184,0.186669,0.003771,-0.127955,...,22748000.0,5.254100e+07,0.361373,42.361908,95.528800,1010.037427,8.616231,3.820837,-18.668990,-0.124605
28874,BODY,2019-12-31,FY,2.482628,0.106085,0.138020,0.059807,0.136532,-0.035618,-0.077851,...,NaN,NaN,0.733111,28.661105,68.946315,497.878129,12.735029,5.293974,-2.978419,-0.078213


## Alternative method: Import data from CSV files

In [54]:
# Import data from CSV files:
df_ratios_2016 = pd.read_csv('Data/key_metrics_2016.csv')
df_ratios_2017 = pd.read_csv('Data/key_metrics_2017.csv')
df_ratios_2018 = pd.read_csv('Data/key_metrics_2018.csv')
df_ratios_2019 = pd.read_csv('Data/key_metrics_2019.csv')
df_ratios_2020 = pd.read_csv('Data/key_metrics_2020.csv')
df_ratios_2021 = pd.read_csv('Data/key_metrics_2021.csv')
df_ratios_2022 = pd.read_csv('Data/key_metrics_2022.csv')

In [55]:
# Combine into one dataframe:
df_ratios_all = pd.concat([df_ratios_2016, df_ratios_2017, df_ratios_2018, df_ratios_2019, df_ratios_2020, df_ratios_2021, df_ratios_2022])
df_ratios_all

,symbol,date,period,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,freeCashFlowPerShare,cashPerShare,bookValuePerShare,tangibleBookValuePerShare,...,averagePayables,averageInventory,daysSalesOutstanding,daysPayablesOutstanding,daysOfInventoryOnHand,receivablesTurnover,payablesTurnover,inventoryTurnover,roe,capexPerShare
0,COD.L,12/31/2016,FY,70.485554,2.363762,3.542939,0.800542,6.739698,33.833715,10.452121,...,6.354000e+09,6.236500e+09,0.000000,72.796846,73.674672,NaN,5.013953,4.954213,0.069864,-2.742397
1,600266.SS,12/31/2016,FY,5.146226,0.638889,-0.771420,-0.775001,4.305225,8.703012,9.318897,...,7.922242e+09,6.882082e+10,0.000000,292.576903,2300.007240,NaN,1.247535,0.158695,0.073410,-0.003581
2,BFLBY,12/31/2016,FY,19.222695,1.224324,-1.573161,-1.888517,4.857479,7.461768,3.493349,...,3.637000e+08,4.245000e+07,0.000000,36.960347,3.940416,NaN,9.875448,92.629808,0.164080,-0.315356
3,LOAD.L,12/31/2016,FY,0.990908,-0.262370,0.032453,0.025662,0.154306,1.740733,1.153587,...,3.449500e+06,4.028500e+06,0.000000,82.774517,87.557156,NaN,4.409570,4.168706,-0.150724,-0.006790
4,FON.WA,12/31/2016,FY,0.390117,-0.526283,-0.014098,-0.045865,4.245777,11.723849,11.723849,...,7.357689e+03,NaN,0.000000,44.628821,1439.279476,NaN,8.178571,0.253599,-0.044890,-0.031766
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20168,IRIX,1/1/2022,FY,3.495428,-0.338824,0.532845,0.519032,1.546722,1.510862,1.305298,...,2.772000e+06,7.614000e+06,65.791143,32.562436,89.440976,5.547859,11.209235,4.080904,-0.224259,-0.013812
20169,USFD,1/1/2022,FY,132.905440,0.739190,1.888540,0.653552,0.667074,19.241473,-9.852860,...,1.662000e+09,1.686000e+09,19.978635,24.429365,24.782136,18.269517,14.941035,14.728351,0.038416,-1.234988
20170,HNI,1/1/2022,FY,50.287951,1.376997,3.030342,1.498319,1.235370,13.574382,2.727220,...,2.337590e+08,1.815910e+08,39.624846,59.789184,46.446031,9.211392,6.104783,7.858583,0.101441,-1.532023
20171,HLIO,1/1/2022,FY,26.906420,3.237865,3.504272,2.597480,0.883482,21.946601,-5.068474,...,8.530100e+07,1.656290e+08,57.666544,55.959713,108.657006,6.329493,6.522550,3.359194,0.147534,-0.906792


In [56]:
# Export clean prices to csv
df_ratios_all.to_csv('Ratios_All_v2.csv')

In [57]:
# Keep important financial columns:
cols_ratios = ['symbol', 'date', 'period', 'revenuePerShare', 'netIncomePerShare', 'operatingCashFlowPerShare', 'marketCap',
        'peRatio', 'interestCoverage', 'dividendYield']          
df_ratios2 = df_ratios_all[cols_ratios]
df_ratios2

,symbol,date,period,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,marketCap,peRatio,interestCoverage,dividendYield
0,COD.L,12/31/2016,FY,70.485554,2.363762,3.542939,2.539902e+10,19.373774,3.477064,0.026773
1,600266.SS,12/31/2016,FY,5.146226,0.638889,-0.771420,2.075904e+10,14.380435,11.610393,0.094187
2,BFLBY,12/31/2016,FY,19.222695,1.224324,-1.573161,1.418396e+09,5.241670,-1.951977,NaN
3,LOAD.L,12/31/2016,FY,0.990908,-0.262370,0.032453,3.324590e+07,-5.278802,-9.047882,NaN
4,FON.WA,12/31/2016,FY,0.390117,-0.526283,-0.014098,2.422499e+07,-36.178257,49.500000,NaN
...,...,...,...,...,...,...,...,...,...,...
20168,IRIX,1/1/2022,FY,3.495428,-0.338824,0.532845,8.142288e+07,-15.583327,NaN,NaN
20169,USFD,1/1/2022,FY,132.905440,0.739190,1.888540,7.822943e+09,47.700869,1.990610,0.003579
20170,HNI,1/1/2022,FY,50.287951,1.376997,3.030342,1.821790e+09,30.457579,12.822452,0.029504
20171,HLIO,1/1/2022,FY,26.906420,3.237865,3.504272,2.475455e+09,23.666827,8.850039,0.004690


In [58]:
# Add year column:
df_ratios3 = df_ratios2.copy()
df_ratios3['Year'] = pd.DatetimeIndex(df_ratios3['date']).year

df_ratios3

,symbol,date,period,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,marketCap,peRatio,interestCoverage,dividendYield,Year
0,COD.L,12/31/2016,FY,70.485554,2.363762,3.542939,2.539902e+10,19.373774,3.477064,0.026773,2016
1,600266.SS,12/31/2016,FY,5.146226,0.638889,-0.771420,2.075904e+10,14.380435,11.610393,0.094187,2016
2,BFLBY,12/31/2016,FY,19.222695,1.224324,-1.573161,1.418396e+09,5.241670,-1.951977,NaN,2016
3,LOAD.L,12/31/2016,FY,0.990908,-0.262370,0.032453,3.324590e+07,-5.278802,-9.047882,NaN,2016
4,FON.WA,12/31/2016,FY,0.390117,-0.526283,-0.014098,2.422499e+07,-36.178257,49.500000,NaN,2016
...,...,...,...,...,...,...,...,...,...,...,...
20168,IRIX,1/1/2022,FY,3.495428,-0.338824,0.532845,8.142288e+07,-15.583327,NaN,NaN,2022
20169,USFD,1/1/2022,FY,132.905440,0.739190,1.888540,7.822943e+09,47.700869,1.990610,0.003579,2022
20170,HNI,1/1/2022,FY,50.287951,1.376997,3.030342,1.821790e+09,30.457579,12.822452,0.029504,2022
20171,HLIO,1/1/2022,FY,26.906420,3.237865,3.504272,2.475455e+09,23.666827,8.850039,0.004690,2022


In [59]:
# add a key/index column consisting of symbol + year
df_ratios3['Key'] = df_ratios3['symbol'].astype(str) + "-" + df_ratios3['Year'].astype(str)
df_ratios3

,symbol,date,period,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,marketCap,peRatio,interestCoverage,dividendYield,Year,Key
0,COD.L,12/31/2016,FY,70.485554,2.363762,3.542939,2.539902e+10,19.373774,3.477064,0.026773,2016,COD.L-2016
1,600266.SS,12/31/2016,FY,5.146226,0.638889,-0.771420,2.075904e+10,14.380435,11.610393,0.094187,2016,600266.SS-2016
2,BFLBY,12/31/2016,FY,19.222695,1.224324,-1.573161,1.418396e+09,5.241670,-1.951977,NaN,2016,BFLBY-2016
3,LOAD.L,12/31/2016,FY,0.990908,-0.262370,0.032453,3.324590e+07,-5.278802,-9.047882,NaN,2016,LOAD.L-2016
4,FON.WA,12/31/2016,FY,0.390117,-0.526283,-0.014098,2.422499e+07,-36.178257,49.500000,NaN,2016,FON.WA-2016
...,...,...,...,...,...,...,...,...,...,...,...,...
20168,IRIX,1/1/2022,FY,3.495428,-0.338824,0.532845,8.142288e+07,-15.583327,NaN,NaN,2022,IRIX-2022
20169,USFD,1/1/2022,FY,132.905440,0.739190,1.888540,7.822943e+09,47.700869,1.990610,0.003579,2022,USFD-2022
20170,HNI,1/1/2022,FY,50.287951,1.376997,3.030342,1.821790e+09,30.457579,12.822452,0.029504,2022,HNI-2022
20171,HLIO,1/1/2022,FY,26.906420,3.237865,3.504272,2.475455e+09,23.666827,8.850039,0.004690,2022,HLIO-2022


In [60]:
cols_keep = ['period', 'revenuePerShare', 'netIncomePerShare', 'operatingCashFlowPerShare', 'marketCap',
        'peRatio', 'interestCoverage', 'dividendYield', 'Key']
df_ratios4 = df_ratios3.copy()
df_ratios4 = df_ratios4[cols_keep]
df_ratios4

,period,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,marketCap,peRatio,interestCoverage,dividendYield,Key
0,FY,70.485554,2.363762,3.542939,2.539902e+10,19.373774,3.477064,0.026773,COD.L-2016
1,FY,5.146226,0.638889,-0.771420,2.075904e+10,14.380435,11.610393,0.094187,600266.SS-2016
2,FY,19.222695,1.224324,-1.573161,1.418396e+09,5.241670,-1.951977,NaN,BFLBY-2016
3,FY,0.990908,-0.262370,0.032453,3.324590e+07,-5.278802,-9.047882,NaN,LOAD.L-2016
4,FY,0.390117,-0.526283,-0.014098,2.422499e+07,-36.178257,49.500000,NaN,FON.WA-2016
...,...,...,...,...,...,...,...,...,...
20168,FY,3.495428,-0.338824,0.532845,8.142288e+07,-15.583327,NaN,NaN,IRIX-2022
20169,FY,132.905440,0.739190,1.888540,7.822943e+09,47.700869,1.990610,0.003579,USFD-2022
20170,FY,50.287951,1.376997,3.030342,1.821790e+09,30.457579,12.822452,0.029504,HNI-2022
20171,FY,26.906420,3.237865,3.504272,2.475455e+09,23.666827,8.850039,0.004690,HLIO-2022


## Join stock pricing data with financial ratios data 

In [61]:
# Join prices with financial ratios dataframe
data = pd.merge(df_prices_clean, df_ratios4, how='inner', on = 'Key')
data

,symbol,date,close,adjClose,Year,PY_adjClose,Offset_symbol,Key,Return,period,revenuePerShare,netIncomePerShare,operatingCashFlowPerShare,marketCap,peRatio,interestCoverage,dividendYield
0,ZYXI,12/30/2022,13.910000,13.910000,2022,8.964234,ZYXI,ZYXI-2022,0.551722,FY,4.111758,0.443185,0.357345,5.454621e+08,31.995663,52.131818,0.006624
1,ZYXI,12/31/2021,9.063600,8.964234,2021,12.102167,ZYXI,ZYXI-2021,-0.259287,FY,3.400606,0.446355,0.181356,2.831626e+08,16.556313,235.431579,0.000004
2,ZYXI,12/31/2020,12.236400,12.102167,2020,7.076080,ZYXI,ZYXI-2020,0.710293,FY,2.150586,0.243559,0.021956,6.120128e+08,67.446863,539.421053,NaN
3,ZYXI,12/31/2019,7.154500,7.076080,2019,2.643415,ZYXI,ZYXI-2019,1.676871,FY,1.274336,0.266010,0.176639,3.156315e+08,33.252370,2213.200000,0.007167
4,ZYXI,12/31/2018,2.644294,2.643415,2018,2.783678,ZYXI,ZYXI-2018,-0.050388,FY,0.892701,0.267164,0.263137,1.251366e+08,13.100560,67.337662,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30008,A,12/31/2020,118.490000,116.890472,2020,83.618065,A,A-2020,0.397909,FY,17.278317,2.326861,2.980583,3.612210e+10,50.239361,10.846154,0.006146
30009,A,12/31/2019,85.310000,83.618065,2019,65.562431,A,A-2019,0.275396,FY,16.034161,3.326087,3.170807,2.600794e+10,24.283790,12.716216,0.007921
30010,A,12/31/2018,67.460000,65.562431,2018,64.488472,A,A-2018,0.016654,FY,15.073620,0.969325,3.334356,2.358610e+10,74.639555,12.373333,0.008098
30011,A,12/29/2017,66.970000,64.488472,2017,43.479897,A,A-2017,0.483179,FY,13.429429,2.054054,2.669670,2.305692e+10,33.708946,10.645570,0.007373


In [62]:
# Create prior year financial ratios columns:
data2 = data.copy()
data2[['PY_revenuePerShare', 'PY_netIncomePerShare',	'PY_operatingCashFlowPerShare',	'PY_marketCap',
      	'PY_peRatio',	'PY_interestCoverage',	'PY_dividendYield']] = 0 

data2

,symbol,date,close,adjClose,Year,PY_adjClose,Offset_symbol,Key,Return,period,...,peRatio,interestCoverage,dividendYield,PY_revenuePerShare,PY_netIncomePerShare,PY_operatingCashFlowPerShare,PY_marketCap,PY_peRatio,PY_interestCoverage,PY_dividendYield
0,ZYXI,12/30/2022,13.910000,13.910000,2022,8.964234,ZYXI,ZYXI-2022,0.551722,FY,...,31.995663,52.131818,0.006624,0,0,0,0,0,0,0
1,ZYXI,12/31/2021,9.063600,8.964234,2021,12.102167,ZYXI,ZYXI-2021,-0.259287,FY,...,16.556313,235.431579,0.000004,0,0,0,0,0,0,0
2,ZYXI,12/31/2020,12.236400,12.102167,2020,7.076080,ZYXI,ZYXI-2020,0.710293,FY,...,67.446863,539.421053,NaN,0,0,0,0,0,0,0
3,ZYXI,12/31/2019,7.154500,7.076080,2019,2.643415,ZYXI,ZYXI-2019,1.676871,FY,...,33.252370,2213.200000,0.007167,0,0,0,0,0,0,0
4,ZYXI,12/31/2018,2.644294,2.643415,2018,2.783678,ZYXI,ZYXI-2018,-0.050388,FY,...,13.100560,67.337662,NaN,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30008,A,12/31/2020,118.490000,116.890472,2020,83.618065,A,A-2020,0.397909,FY,...,50.239361,10.846154,0.006146,0,0,0,0,0,0,0
30009,A,12/31/2019,85.310000,83.618065,2019,65.562431,A,A-2019,0.275396,FY,...,24.283790,12.716216,0.007921,0,0,0,0,0,0,0
30010,A,12/31/2018,67.460000,65.562431,2018,64.488472,A,A-2018,0.016654,FY,...,74.639555,12.373333,0.008098,0,0,0,0,0,0,0
30011,A,12/29/2017,66.970000,64.488472,2017,43.479897,A,A-2017,0.483179,FY,...,33.708946,10.645570,0.007373,0,0,0,0,0,0,0


In [63]:
# Reset offset_symbol column to properly index prior year financial ratios
data2['Offset_symbol'] = 0
data2

,symbol,date,close,adjClose,Year,PY_adjClose,Offset_symbol,Key,Return,period,...,peRatio,interestCoverage,dividendYield,PY_revenuePerShare,PY_netIncomePerShare,PY_operatingCashFlowPerShare,PY_marketCap,PY_peRatio,PY_interestCoverage,PY_dividendYield
0,ZYXI,12/30/2022,13.910000,13.910000,2022,8.964234,0,ZYXI-2022,0.551722,FY,...,31.995663,52.131818,0.006624,0,0,0,0,0,0,0
1,ZYXI,12/31/2021,9.063600,8.964234,2021,12.102167,0,ZYXI-2021,-0.259287,FY,...,16.556313,235.431579,0.000004,0,0,0,0,0,0,0
2,ZYXI,12/31/2020,12.236400,12.102167,2020,7.076080,0,ZYXI-2020,0.710293,FY,...,67.446863,539.421053,NaN,0,0,0,0,0,0,0
3,ZYXI,12/31/2019,7.154500,7.076080,2019,2.643415,0,ZYXI-2019,1.676871,FY,...,33.252370,2213.200000,0.007167,0,0,0,0,0,0,0
4,ZYXI,12/31/2018,2.644294,2.643415,2018,2.783678,0,ZYXI-2018,-0.050388,FY,...,13.100560,67.337662,NaN,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30008,A,12/31/2020,118.490000,116.890472,2020,83.618065,0,A-2020,0.397909,FY,...,50.239361,10.846154,0.006146,0,0,0,0,0,0,0
30009,A,12/31/2019,85.310000,83.618065,2019,65.562431,0,A-2019,0.275396,FY,...,24.283790,12.716216,0.007921,0,0,0,0,0,0,0
30010,A,12/31/2018,67.460000,65.562431,2018,64.488472,0,A-2018,0.016654,FY,...,74.639555,12.373333,0.008098,0,0,0,0,0,0,0
30011,A,12/29/2017,66.970000,64.488472,2017,43.479897,0,A-2017,0.483179,FY,...,33.708946,10.645570,0.007373,0,0,0,0,0,0,0


In [64]:
j = 1
for i in data2['Offset_symbol']:
    try:
        data2['Offset_symbol'].iloc[j-1] = data2['symbol'].iloc[j]
        j = j+1
    except:
        print('\n' 'Last row, no value to fill!')

data2

C:\Users\willm\AppData\Local\Temp\ipykernel_10212\2579680125.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['Offset_symbol'].iloc[j-1] = data2['symbol'].iloc[j]



Last row, no value to fill!


,symbol,date,close,adjClose,Year,PY_adjClose,Offset_symbol,Key,Return,period,...,peRatio,interestCoverage,dividendYield,PY_revenuePerShare,PY_netIncomePerShare,PY_operatingCashFlowPerShare,PY_marketCap,PY_peRatio,PY_interestCoverage,PY_dividendYield
0,ZYXI,12/30/2022,13.910000,13.910000,2022,8.964234,ZYXI,ZYXI-2022,0.551722,FY,...,31.995663,52.131818,0.006624,0,0,0,0,0,0,0
1,ZYXI,12/31/2021,9.063600,8.964234,2021,12.102167,ZYXI,ZYXI-2021,-0.259287,FY,...,16.556313,235.431579,0.000004,0,0,0,0,0,0,0
2,ZYXI,12/31/2020,12.236400,12.102167,2020,7.076080,ZYXI,ZYXI-2020,0.710293,FY,...,67.446863,539.421053,NaN,0,0,0,0,0,0,0
3,ZYXI,12/31/2019,7.154500,7.076080,2019,2.643415,ZYXI,ZYXI-2019,1.676871,FY,...,33.252370,2213.200000,0.007167,0,0,0,0,0,0,0
4,ZYXI,12/31/2018,2.644294,2.643415,2018,2.783678,ZYXI,ZYXI-2018,-0.050388,FY,...,13.100560,67.337662,NaN,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30008,A,12/31/2020,118.490000,116.890472,2020,83.618065,A,A-2020,0.397909,FY,...,50.239361,10.846154,0.006146,0,0,0,0,0,0,0
30009,A,12/31/2019,85.310000,83.618065,2019,65.562431,A,A-2019,0.275396,FY,...,24.283790,12.716216,0.007921,0,0,0,0,0,0,0
30010,A,12/31/2018,67.460000,65.562431,2018,64.488472,A,A-2018,0.016654,FY,...,74.639555,12.373333,0.008098,0,0,0,0,0,0,0
30011,A,12/29/2017,66.970000,64.488472,2017,43.479897,A,A-2017,0.483179,FY,...,33.708946,10.645570,0.007373,0,0,0,0,0,0,0


In [65]:
# fill prior year financial ratios columns
    ## NA rows to be deleted
j = 0
for i in data2['PY_revenuePerShare']:
    if data2['Offset_symbol'].iloc[j] == data2['symbol'].iloc[j]:
        data2['PY_revenuePerShare'].iloc[j] = data2['revenuePerShare'].iloc[j+1]
        data2['PY_netIncomePerShare'].iloc[j] = data2['netIncomePerShare'].iloc[j+1]
        data2['PY_operatingCashFlowPerShare'].iloc[j] = data2['operatingCashFlowPerShare'].iloc[j+1]
        data2['PY_marketCap'].iloc[j] = data2['marketCap'].iloc[j+1]
        data2['PY_peRatio'].iloc[j] = data2['peRatio'].iloc[j+1]
        data2['PY_interestCoverage'].iloc[j] = data2['interestCoverage'].iloc[j+1]
        data2['PY_dividendYield'].iloc[j] = data2['dividendYield'].iloc[j+1]

    else:
        data2['PY_revenuePerShare'].iloc[j] = "NA"
        data2['PY_netIncomePerShare'].iloc[j] = "NA"
        data2['PY_operatingCashFlowPerShare'].iloc[j] = "NA"
        data2['PY_marketCap'].iloc[j] = "NA"
        data2['PY_peRatio'].iloc[j] = "NA"
        data2['PY_interestCoverage'].iloc[j] = "NA"
        data2['PY_dividendYield'].iloc[j] = "NA"
        
    j = j+1

data2

C:\Users\willm\AppData\Local\Temp\ipykernel_10212\211383028.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['PY_revenuePerShare'].iloc[j] = data2['revenuePerShare'].iloc[j+1]
C:\Users\willm\AppData\Local\Temp\ipykernel_10212\211383028.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['PY_netIncomePerShare'].iloc[j] = data2['netIncomePerShare'].iloc[j+1]
C:\Users\willm\AppData\Local\Temp\ipykernel_10212\211383028.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/

,symbol,date,close,adjClose,Year,PY_adjClose,Offset_symbol,Key,Return,period,...,peRatio,interestCoverage,dividendYield,PY_revenuePerShare,PY_netIncomePerShare,PY_operatingCashFlowPerShare,PY_marketCap,PY_peRatio,PY_interestCoverage,PY_dividendYield
0,ZYXI,12/30/2022,13.910000,13.910000,2022,8.964234,ZYXI,ZYXI-2022,0.551722,FY,...,31.995663,52.131818,0.006624,3.400606,0.446355,0.181356,283162618.8,16.556313,235.431579,0.000004
1,ZYXI,12/31/2021,9.063600,8.964234,2021,12.102167,ZYXI,ZYXI-2021,-0.259287,FY,...,16.556313,235.431579,0.000004,2.150586,0.243559,0.021956,612012832.7,67.446863,539.421053,NaN
2,ZYXI,12/31/2020,12.236400,12.102167,2020,7.076080,ZYXI,ZYXI-2020,0.710293,FY,...,67.446863,539.421053,NaN,1.274336,0.26601,0.176639,315631493.4,33.25237,2213.2,0.007167
3,ZYXI,12/31/2019,7.154500,7.076080,2019,2.643415,ZYXI,ZYXI-2019,1.676871,FY,...,33.252370,2213.200000,0.007167,0.892701,0.267164,0.263137,125136550.0,13.10056,67.337662,NaN
4,ZYXI,12/31/2018,2.644294,2.643415,2018,2.783678,ZYXI,ZYXI-2018,-0.050388,FY,...,13.100560,67.337662,NaN,0.662452,0.208218,0.233521,154991906.5,21.044386,6.168276,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30008,A,12/31/2020,118.490000,116.890472,2020,83.618065,A,A-2020,0.397909,FY,...,50.239361,10.846154,0.006146,16.034161,3.326087,3.170807,26007938905.0,24.28379,12.716216,0.007921
30009,A,12/31/2019,85.310000,83.618065,2019,65.562431,A,A-2019,0.275396,FY,...,24.283790,12.716216,0.007921,15.07362,0.969325,3.334356,23586099511.0,74.639555,12.373333,0.008098
30010,A,12/31/2018,67.460000,65.562431,2018,64.488472,A,A-2018,0.016654,FY,...,74.639555,12.373333,0.008098,13.429429,2.054054,2.66967,23056919301.0,33.708946,10.64557,0.007373
30011,A,12/29/2017,66.970000,64.488472,2017,43.479897,A,A-2017,0.483179,FY,...,33.708946,10.645570,0.007373,12.618619,1.387387,2.381381,14645339834.0,31.69987,8.541667,0.010242


In [66]:
# Export clean prices to csv
data2.to_csv('Data2.csv')

In [67]:
# Check for duplicates
# Check symbols with value counts equal to 1 (only 1 year of data)
print('Count of symbols = 1:', (data2['Key'].value_counts() == 1).sum())
print('Count of symbols > 1:', (data2['Key'].value_counts() > 1).sum())


Count of symbols = 1: 29673
Count of symbols > 1: 170


In [68]:
# Remove duplicates where Key values counts >1
data3 = data2.copy()
data3 = data3[data3.groupby('Key').Key.transform('count')==1]
data3

,symbol,date,close,adjClose,Year,PY_adjClose,Offset_symbol,Key,Return,period,...,peRatio,interestCoverage,dividendYield,PY_revenuePerShare,PY_netIncomePerShare,PY_operatingCashFlowPerShare,PY_marketCap,PY_peRatio,PY_interestCoverage,PY_dividendYield
0,ZYXI,12/30/2022,13.910000,13.910000,2022,8.964234,ZYXI,ZYXI-2022,0.551722,FY,...,31.995663,52.131818,0.006624,3.400606,0.446355,0.181356,283162618.8,16.556313,235.431579,0.000004
1,ZYXI,12/31/2021,9.063600,8.964234,2021,12.102167,ZYXI,ZYXI-2021,-0.259287,FY,...,16.556313,235.431579,0.000004,2.150586,0.243559,0.021956,612012832.7,67.446863,539.421053,NaN
2,ZYXI,12/31/2020,12.236400,12.102167,2020,7.076080,ZYXI,ZYXI-2020,0.710293,FY,...,67.446863,539.421053,NaN,1.274336,0.26601,0.176639,315631493.4,33.25237,2213.2,0.007167
3,ZYXI,12/31/2019,7.154500,7.076080,2019,2.643415,ZYXI,ZYXI-2019,1.676871,FY,...,33.252370,2213.200000,0.007167,0.892701,0.267164,0.263137,125136550.0,13.10056,67.337662,NaN
4,ZYXI,12/31/2018,2.644294,2.643415,2018,2.783678,ZYXI,ZYXI-2018,-0.050388,FY,...,13.100560,67.337662,NaN,0.662452,0.208218,0.233521,154991906.5,21.044386,6.168276,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30008,A,12/31/2020,118.490000,116.890472,2020,83.618065,A,A-2020,0.397909,FY,...,50.239361,10.846154,0.006146,16.034161,3.326087,3.170807,26007938905.0,24.28379,12.716216,0.007921
30009,A,12/31/2019,85.310000,83.618065,2019,65.562431,A,A-2019,0.275396,FY,...,24.283790,12.716216,0.007921,15.07362,0.969325,3.334356,23586099511.0,74.639555,12.373333,0.008098
30010,A,12/31/2018,67.460000,65.562431,2018,64.488472,A,A-2018,0.016654,FY,...,74.639555,12.373333,0.008098,13.429429,2.054054,2.66967,23056919301.0,33.708946,10.64557,0.007373
30011,A,12/29/2017,66.970000,64.488472,2017,43.479897,A,A-2017,0.483179,FY,...,33.708946,10.645570,0.007373,12.618619,1.387387,2.381381,14645339834.0,31.69987,8.541667,0.010242


In [69]:
# Check for duplicates
# Check symbols with value counts equal to 1 (only 1 year of data)
print('Count of symbols = 1:', (data3['Key'].value_counts() == 1).sum())
print('Count of symbols > 1:', (data3['Key'].value_counts() > 1).sum())


Count of symbols = 1: 29673
Count of symbols > 1: 0


In [70]:
# Drop unnecessary columns:
data4 = data3.copy()
data4 = data3.drop(['revenuePerShare','netIncomePerShare','operatingCashFlowPerShare','marketCap', 'peRatio',
             'interestCoverage', 'dividendYield'], axis=1)
data4

,symbol,date,close,adjClose,Year,PY_adjClose,Offset_symbol,Key,Return,period,PY_revenuePerShare,PY_netIncomePerShare,PY_operatingCashFlowPerShare,PY_marketCap,PY_peRatio,PY_interestCoverage,PY_dividendYield
0,ZYXI,12/30/2022,13.910000,13.910000,2022,8.964234,ZYXI,ZYXI-2022,0.551722,FY,3.400606,0.446355,0.181356,283162618.8,16.556313,235.431579,0.000004
1,ZYXI,12/31/2021,9.063600,8.964234,2021,12.102167,ZYXI,ZYXI-2021,-0.259287,FY,2.150586,0.243559,0.021956,612012832.7,67.446863,539.421053,NaN
2,ZYXI,12/31/2020,12.236400,12.102167,2020,7.076080,ZYXI,ZYXI-2020,0.710293,FY,1.274336,0.26601,0.176639,315631493.4,33.25237,2213.2,0.007167
3,ZYXI,12/31/2019,7.154500,7.076080,2019,2.643415,ZYXI,ZYXI-2019,1.676871,FY,0.892701,0.267164,0.263137,125136550.0,13.10056,67.337662,NaN
4,ZYXI,12/31/2018,2.644294,2.643415,2018,2.783678,ZYXI,ZYXI-2018,-0.050388,FY,0.662452,0.208218,0.233521,154991906.5,21.044386,6.168276,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30008,A,12/31/2020,118.490000,116.890472,2020,83.618065,A,A-2020,0.397909,FY,16.034161,3.326087,3.170807,26007938905.0,24.28379,12.716216,0.007921
30009,A,12/31/2019,85.310000,83.618065,2019,65.562431,A,A-2019,0.275396,FY,15.07362,0.969325,3.334356,23586099511.0,74.639555,12.373333,0.008098
30010,A,12/31/2018,67.460000,65.562431,2018,64.488472,A,A-2018,0.016654,FY,13.429429,2.054054,2.66967,23056919301.0,33.708946,10.64557,0.007373
30011,A,12/29/2017,66.970000,64.488472,2017,43.479897,A,A-2017,0.483179,FY,12.618619,1.387387,2.381381,14645339834.0,31.69987,8.541667,0.010242


In [71]:
profile = ("https://financialmodelingprep.com/api/v3/profile/AAPL?apikey=4f3f56682d5bbab7a42251bad1821ade")

In [72]:
tickers_list

['ZYXI',
 'ZYME',
 'ZXZZT',
 'ZXYZ-A',
 'ZWS',
 'ZVIA',
 'ZUO',
 'ZUMZ',
 'ZTST',
 'ZTS',
 'ZTO',
 'ZTEK',
 'ZS',
 'ZNTL',
 'ZM',
 'ZLAB',
 'ZKIN',
 'ZJZZT',
 'ZIVOW',
 'ZIVO',
 'ZIP',
 'ZIONP',
 'ZIONO',
 'ZIONL',
 'ZION',
 'ZIM',
 'ZI',
 'ZH',
 'ZGN',
 'ZG',
 'ZFOXW',
 'ZFOX',
 'ZEV',
 'ZEUS',
 'ZETA',
 'ZEPP',
 'ZENV',
 'ZD',
 'ZCMD',
 'ZBRA',
 'ZBH',
 'Z',
 'YY',
 'YUMC',
 'YUM',
 'YTRA',
 'YTEN',
 'YSG',
 'YS',
 'YRD',
 'YQ',
 'YPF',
 'YOU',
 'YORW',
 'YMM',
 'YMAB',
 'YJ',
 'YI',
 'YGMZ',
 'YEXT',
 'YETI',
 'YELP',
 'YALA',
 'XYL',
 'XYF',
 'XXII',
 'XWEL',
 'XTLB',
 'XRX',
 'XRTX',
 'XRAY',
 'XPRO',
 'XPOF',
 'XPO',
 'XPEV',
 'XPER',
 'XPEL',
 'XPDBU',
 'XP',
 'XOSWW',
 'XOS',
 'XOMAP',
 'XOMAO',
 'XOMA',
 'XOM',
 'XNET',
 'XNCR',
 'XMTR',
 'XLO',
 'XIN',
 'XHR',
 'XGN',
 'XFOR',
 'XFINW',
 'XFINU',
 'XFIN',
 'XERS',
 'XENE',
 'XELB',
 'XELA',
 'XEL',
 'XCUR',
 'XBIT',
 'XBIO',
 'XAIR',
 'X',
 'WYNN',
 'WY',
 'WWW',
 'WWD',
 'WW',
 'WVVIP',
 'WVVI',
 'WVE',
 'WULF',
 'WU',
 'WTW

## Pull sector data for each stock ticker
- Note: subscription may be required
- Alternative method: Manually upload sector information from csv file(s):

In [ ]:
## Get company profile / sector data from stocks on tickers_list ##

base_profile = ("https://financialmodelingprep.com/api/v3/profile/")
end_profile = ("?apikey=4f3f56682d5bbab7a42251bad1821ade")     

json_list_profile = []
for i in tickers_list:
    url_profile = base_profile + i + end_profile
    #print(get_jsonparsed_data(url))
    #print(url)
    json_profile = pd.read_json(url_profile)
    json_list_profile.append(json_profile)
   

json_list_profile


In [238]:
# Convert pricing JSON files into dataframe 
df_profile = pd.concat(json_list_profile, axis=0, ignore_index=True)

,symbol,price,beta,volAvg,mktCap,lastDiv,range,changes,companyName,currency,...,zip,dcfDiff,dcf,image,ipoDate,defaultImage,isEtf,isActivelyTrading,isAdr,isFund
0,REGN,828.7250,0.209400,650534,89093738575,0.00,538.01-837.55,6.6349,"Regeneron Pharmaceuticals, Inc.",USD,...,10591-6707,21.220400,764.05000,https://financialmodelingprep.com/image-stock/...,1991-01-01,False,False,True,False,False
1,AMOT,34.9150,1.481234,65657,561475098,0.10,21.14-44.88,-1.3850,Allied Motion Technologies Inc.,USD,...,14228,1.672720,37.40730,https://financialmodelingprep.com/image-stock/...,1980-10-20,False,False,True,False,False
2,RELX,32.6250,0.480077,817552,62139205728,0.65,23.39-32.885,0.0150,RELX PLC,USD,...,WC2N 5JR,1.781340,27.62870,https://financialmodelingprep.com/image-stock/...,1994-10-06,False,False,True,True,False
3,AMRN,1.3500,1.881795,4835590,548256600,0.00,1.04-3.74,-0.0800,Amarin Corporation plc,USD,...,2,0.125150,1.72485,https://financialmodelingprep.com/image-stock/...,1993-04-01,False,False,True,True,False
4,RES,7.8750,1.819010,1394749,1712812500,0.16,5.7-12.91,-0.1150,"RPC, Inc.",USD,...,30329,0.332271,9.78227,https://financialmodelingprep.com/image-stock/...,1984-07-02,False,False,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6853,BNNR,10.2400,0.000000,76055,200959990,0.00,9.795-10.39,0.0100,Banner Acquisition Corp.,USD,...,84043,NaN,0.00000,https://financialmodelingprep.com/image-stock/...,2021-10-29,False,False,True,False,False
6854,BNNRU,10.2400,0.000000,4622,194610340,0.00,8.93-10.415,0.0000,Banner Acquisition Corp.,USD,...,84043,NaN,0.00000,https://financialmodelingprep.com/image-stock/...,2021-09-08,False,False,True,False,False
6855,BNNRW,0.0400,0.000000,0,0,0.00,0.04-0.04,0.0000,Banner Acquisition Corp.,NaN,...,NaN,NaN,0.00000,https://financialmodelingprep.com/image-stock/...,,True,False,True,False,False
6856,BOCNU,10.2800,0.000000,2691,242204417,0.00,9.9-10.28,0.0400,Blue Ocean Acquisition Corp.,USD,...,20815,NaN,0.00000,https://financialmodelingprep.com/image-stock/...,,False,False,True,False,False


In [262]:
df_profile['sector'].value_counts()

Financial Services        1723
Healthcare                1257
Technology                 788
Industrials                707
Consumer Cyclical          574
                           375
Real Estate                354
Communication Services     282
Consumer Defensive         237
Energy                     230
Basic Materials            206
Utilities                  125
Name: sector, dtype: int64

In [304]:
# Get dataframe with symbol and sector only
    
df_sector = df_profile[['symbol', 'sector']]
df_sector

,symbol,sector
0,REGN,Healthcare
1,AMOT,Technology
2,RELX,Communication Services
3,AMRN,Healthcare
4,RES,Energy
...,...,...
6853,BNNR,Financial Services
6854,BNNRU,Financial Services
6855,BNNRW,
6856,BOCNU,Financial Services


In [305]:
df_sector['sector'] == ''


0       False
1       False
2       False
3       False
4       False
        ...  
6853    False
6854    False
6855     True
6856    False
6857    False
Name: sector, Length: 6858, dtype: bool

In [311]:
df_sector['sector'].iloc[6855] == ''

True

In [ ]:
df_sector2 = df_sector.copy()

j = 0
for i in df_sector2['sector']:
    if df_sector2['sector'].iloc[j] == '':
        df_sector2['sector'].iloc[j] = 'NA'
    else:
        print(j)
    j = j+1

df_sector2

In [317]:
# Export to CSV
df_sector2.to_csv('Sector.csv')

## Alternate method: upload sector data from csv file

In [75]:
df_sector3 = pd.read_csv('Data/Sector.csv')
df_sector3

,Unnamed: 0,symbol,sector
0,0,REGN,Healthcare
1,1,AMOT,Technology
2,2,RELX,Communication Services
3,3,AMRN,Healthcare
4,4,RES,Energy
...,...,...,...
6853,6853,BNNR,Financial Services
6854,6854,BNNRU,Financial Services
6855,6855,BNNRW,NaN
6856,6856,BOCNU,Financial Services


In [76]:
# Join sector data with pricing & financial ratio data:
stock_data = pd.merge(data4, df_sector3, how='inner', on = 'symbol')
stock_data

,symbol,date,close,adjClose,Year,PY_adjClose,Offset_symbol,Key,Return,period,PY_revenuePerShare,PY_netIncomePerShare,PY_operatingCashFlowPerShare,PY_marketCap,PY_peRatio,PY_interestCoverage,PY_dividendYield,Unnamed: 0,sector
0,ZYXI,12/30/2022,13.910000,13.910000,2022,8.964234,ZYXI,ZYXI-2022,0.551722,FY,3.400606,0.446355,0.181356,283162618.8,16.556313,235.431579,0.000004,637,Healthcare
1,ZYXI,12/31/2021,9.063600,8.964234,2021,12.102167,ZYXI,ZYXI-2021,-0.259287,FY,2.150586,0.243559,0.021956,612012832.7,67.446863,539.421053,NaN,637,Healthcare
2,ZYXI,12/31/2020,12.236400,12.102167,2020,7.076080,ZYXI,ZYXI-2020,0.710293,FY,1.274336,0.26601,0.176639,315631493.4,33.25237,2213.2,0.007167,637,Healthcare
3,ZYXI,12/31/2019,7.154500,7.076080,2019,2.643415,ZYXI,ZYXI-2019,1.676871,FY,0.892701,0.267164,0.263137,125136550.0,13.10056,67.337662,NaN,637,Healthcare
4,ZYXI,12/31/2018,2.644294,2.643415,2018,2.783678,ZYXI,ZYXI-2018,-0.050388,FY,0.662452,0.208218,0.233521,154991906.5,21.044386,6.168276,NaN,637,Healthcare
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29613,A,12/31/2020,118.490000,116.890472,2020,83.618065,A,A-2020,0.397909,FY,16.034161,3.326087,3.170807,26007938905.0,24.28379,12.716216,0.007921,2520,Healthcare
29614,A,12/31/2019,85.310000,83.618065,2019,65.562431,A,A-2019,0.275396,FY,15.07362,0.969325,3.334356,23586099511.0,74.639555,12.373333,0.008098,2520,Healthcare
29615,A,12/31/2018,67.460000,65.562431,2018,64.488472,A,A-2018,0.016654,FY,13.429429,2.054054,2.66967,23056919301.0,33.708946,10.64557,0.007373,2520,Healthcare
29616,A,12/29/2017,66.970000,64.488472,2017,43.479897,A,A-2017,0.483179,FY,12.618619,1.387387,2.381381,14645339834.0,31.69987,8.541667,0.010242,2520,Healthcare


In [77]:
# S&P 500 returns manually sourced from Yahoo Finance:
    ## https://finance.yahoo.com/quote/SPY/performance?p=SPY
    # 2016 = 12.00%
    # 2017 = 21.70%
    # 2018 = -4.56%
    # 2019 = 31.22%
    # 2020 = 18.37%
    # 2021 = 28.75%
    # 2022 = -18.17%

SP_data = [[2016, 0.1200], [2017, 0.2170], [2018, -0.0456], [2019, 0.3122], [2020, 0.1837], [2021, 0.2875], [2022, -0.1817]]
df_SP = pd.DataFrame(SP_data, columns=['Year', 'S&P500_return'])
df_SP

,Year,S&P500_return
0,2016,0.1200
1,2017,0.2170
2,2018,-0.0456
3,2019,0.3122
4,2020,0.1837
5,2021,0.2875
6,2022,-0.1817


In [78]:
# Pull S&P500 returns into stock dataframe:
stock_data2 = pd.merge(left = stock_data, right = df_SP, how='left', on = 'Year')
stock_data2

,symbol,date,close,adjClose,Year,PY_adjClose,Offset_symbol,Key,Return,period,PY_revenuePerShare,PY_netIncomePerShare,PY_operatingCashFlowPerShare,PY_marketCap,PY_peRatio,PY_interestCoverage,PY_dividendYield,Unnamed: 0,sector,S&P500_return
0,ZYXI,12/30/2022,13.910000,13.910000,2022,8.964234,ZYXI,ZYXI-2022,0.551722,FY,3.400606,0.446355,0.181356,283162618.8,16.556313,235.431579,0.000004,637,Healthcare,-0.1817
1,ZYXI,12/31/2021,9.063600,8.964234,2021,12.102167,ZYXI,ZYXI-2021,-0.259287,FY,2.150586,0.243559,0.021956,612012832.7,67.446863,539.421053,NaN,637,Healthcare,0.2875
2,ZYXI,12/31/2020,12.236400,12.102167,2020,7.076080,ZYXI,ZYXI-2020,0.710293,FY,1.274336,0.26601,0.176639,315631493.4,33.25237,2213.2,0.007167,637,Healthcare,0.1837
3,ZYXI,12/31/2019,7.154500,7.076080,2019,2.643415,ZYXI,ZYXI-2019,1.676871,FY,0.892701,0.267164,0.263137,125136550.0,13.10056,67.337662,NaN,637,Healthcare,0.3122
4,ZYXI,12/31/2018,2.644294,2.643415,2018,2.783678,ZYXI,ZYXI-2018,-0.050388,FY,0.662452,0.208218,0.233521,154991906.5,21.044386,6.168276,NaN,637,Healthcare,-0.0456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29613,A,12/31/2020,118.490000,116.890472,2020,83.618065,A,A-2020,0.397909,FY,16.034161,3.326087,3.170807,26007938905.0,24.28379,12.716216,0.007921,2520,Healthcare,0.1837
29614,A,12/31/2019,85.310000,83.618065,2019,65.562431,A,A-2019,0.275396,FY,15.07362,0.969325,3.334356,23586099511.0,74.639555,12.373333,0.008098,2520,Healthcare,0.3122
29615,A,12/31/2018,67.460000,65.562431,2018,64.488472,A,A-2018,0.016654,FY,13.429429,2.054054,2.66967,23056919301.0,33.708946,10.64557,0.007373,2520,Healthcare,-0.0456
29616,A,12/29/2017,66.970000,64.488472,2017,43.479897,A,A-2017,0.483179,FY,12.618619,1.387387,2.381381,14645339834.0,31.69987,8.541667,0.010242,2520,Healthcare,0.2170


In [79]:
# Drop unnecessary columns:
stock_data3 = stock_data2.copy()
stock_data3 = stock_data3.drop(columns=['Offset_symbol'])
stock_data3

,symbol,date,close,adjClose,Year,PY_adjClose,Key,Return,period,PY_revenuePerShare,PY_netIncomePerShare,PY_operatingCashFlowPerShare,PY_marketCap,PY_peRatio,PY_interestCoverage,PY_dividendYield,Unnamed: 0,sector,S&P500_return
0,ZYXI,12/30/2022,13.910000,13.910000,2022,8.964234,ZYXI-2022,0.551722,FY,3.400606,0.446355,0.181356,283162618.8,16.556313,235.431579,0.000004,637,Healthcare,-0.1817
1,ZYXI,12/31/2021,9.063600,8.964234,2021,12.102167,ZYXI-2021,-0.259287,FY,2.150586,0.243559,0.021956,612012832.7,67.446863,539.421053,NaN,637,Healthcare,0.2875
2,ZYXI,12/31/2020,12.236400,12.102167,2020,7.076080,ZYXI-2020,0.710293,FY,1.274336,0.26601,0.176639,315631493.4,33.25237,2213.2,0.007167,637,Healthcare,0.1837
3,ZYXI,12/31/2019,7.154500,7.076080,2019,2.643415,ZYXI-2019,1.676871,FY,0.892701,0.267164,0.263137,125136550.0,13.10056,67.337662,NaN,637,Healthcare,0.3122
4,ZYXI,12/31/2018,2.644294,2.643415,2018,2.783678,ZYXI-2018,-0.050388,FY,0.662452,0.208218,0.233521,154991906.5,21.044386,6.168276,NaN,637,Healthcare,-0.0456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29613,A,12/31/2020,118.490000,116.890472,2020,83.618065,A-2020,0.397909,FY,16.034161,3.326087,3.170807,26007938905.0,24.28379,12.716216,0.007921,2520,Healthcare,0.1837
29614,A,12/31/2019,85.310000,83.618065,2019,65.562431,A-2019,0.275396,FY,15.07362,0.969325,3.334356,23586099511.0,74.639555,12.373333,0.008098,2520,Healthcare,0.3122
29615,A,12/31/2018,67.460000,65.562431,2018,64.488472,A-2018,0.016654,FY,13.429429,2.054054,2.66967,23056919301.0,33.708946,10.64557,0.007373,2520,Healthcare,-0.0456
29616,A,12/29/2017,66.970000,64.488472,2017,43.479897,A-2017,0.483179,FY,12.618619,1.387387,2.381381,14645339834.0,31.69987,8.541667,0.010242,2520,Healthcare,0.2170


In [80]:
# Drop NA rows:
stock_data4 = stock_data3.copy()
stock_data4.drop(stock_data4.loc[stock_data4['PY_revenuePerShare'] == 'NA'].index, inplace=True)
stock_data4.drop(stock_data4.loc[stock_data4['Return'] == 'inf'].index, inplace=True)
stock_data4

,symbol,date,close,adjClose,Year,PY_adjClose,Key,Return,period,PY_revenuePerShare,PY_netIncomePerShare,PY_operatingCashFlowPerShare,PY_marketCap,PY_peRatio,PY_interestCoverage,PY_dividendYield,Unnamed: 0,sector,S&P500_return
0,ZYXI,12/30/2022,13.910000,13.910000,2022,8.964234,ZYXI-2022,0.551722,FY,3.400606,0.446355,0.181356,283162618.8,16.556313,235.431579,0.000004,637,Healthcare,-0.1817
1,ZYXI,12/31/2021,9.063600,8.964234,2021,12.102167,ZYXI-2021,-0.259287,FY,2.150586,0.243559,0.021956,612012832.7,67.446863,539.421053,NaN,637,Healthcare,0.2875
2,ZYXI,12/31/2020,12.236400,12.102167,2020,7.076080,ZYXI-2020,0.710293,FY,1.274336,0.26601,0.176639,315631493.4,33.25237,2213.2,0.007167,637,Healthcare,0.1837
3,ZYXI,12/31/2019,7.154500,7.076080,2019,2.643415,ZYXI-2019,1.676871,FY,0.892701,0.267164,0.263137,125136550.0,13.10056,67.337662,NaN,637,Healthcare,0.3122
4,ZYXI,12/31/2018,2.644294,2.643415,2018,2.783678,ZYXI-2018,-0.050388,FY,0.662452,0.208218,0.233521,154991906.5,21.044386,6.168276,NaN,637,Healthcare,-0.0456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29612,A,12/31/2021,159.650000,158.397827,2021,116.890472,A-2021,0.355096,FY,17.278317,2.326861,2.980583,36122100464.0,50.239361,10.846154,0.006146,2520,Healthcare,0.2875
29613,A,12/31/2020,118.490000,116.890472,2020,83.618065,A-2020,0.397909,FY,16.034161,3.326087,3.170807,26007938905.0,24.28379,12.716216,0.007921,2520,Healthcare,0.1837
29614,A,12/31/2019,85.310000,83.618065,2019,65.562431,A-2019,0.275396,FY,15.07362,0.969325,3.334356,23586099511.0,74.639555,12.373333,0.008098,2520,Healthcare,0.3122
29615,A,12/31/2018,67.460000,65.562431,2018,64.488472,A-2018,0.016654,FY,13.429429,2.054054,2.66967,23056919301.0,33.708946,10.64557,0.007373,2520,Healthcare,-0.0456


In [81]:
# Convert financial columns to float data types
stock_data4 = stock_data4.astype({'PY_revenuePerShare' : 'float', 'PY_netIncomePerShare' :'float', 'PY_operatingCashFlowPerShare' : 'float',
                                'PY_marketCap' : 'float', 'PY_peRatio' : 'float', 'PY_interestCoverage' : 'float', 
                                  'PY_dividendYield' : 'float'})
stock_data4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24003 entries, 0 to 29616
Data columns (total 19 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   symbol                        24003 non-null  object 
 1   date                          24003 non-null  object 
 2   close                         24003 non-null  float64
 3   adjClose                      24003 non-null  float64
 4   Year                          24003 non-null  int64  
 5   PY_adjClose                   24003 non-null  float64
 6   Key                           24003 non-null  object 
 7   Return                        24003 non-null  float64
 8   period                        24003 non-null  object 
 9   PY_revenuePerShare            23469 non-null  float64
 10  PY_netIncomePerShare          23469 non-null  float64
 11  PY_operatingCashFlowPerShare  23469 non-null  float64
 12  PY_marketCap                  23469 non-null  float64
 13  P

## Apply market cap categories according to FINRA

In [82]:
# Apply market cap categories according to FINRA: 
    # Micro-cap < 250mm ()
    # Small cap = 250mm to 2B
    # Mid cap = 2B to 10B
    # Large cap = > 10B
# https://www.finra.org/investors/insights/market-cap#:~:text=large%2Dcap%3A%20market%20value%20between,of%20less%20than%20%24250%20million

# Create market cap conditions
conditions = [
    (stock_data4['PY_marketCap'] < 250000000),
    (stock_data4['PY_marketCap'] > 250000000) & (stock_data4['PY_marketCap'] <= 2000000000),
    (stock_data4['PY_marketCap'] > 2000000000) & (stock_data4['PY_marketCap'] <= 10000000000),
    (stock_data4['PY_marketCap'] > 10000000000)
]

# Create market cap values to assign for each condition
values = ['micro-cap', 'small-cap', 'mid-cap', 'large-cap']

# Create new column and assign values
stock_data4['MarketCap_Tier'] = np.select(conditions, values)

stock_data4

,symbol,date,close,adjClose,Year,PY_adjClose,Key,Return,period,PY_revenuePerShare,PY_netIncomePerShare,PY_operatingCashFlowPerShare,PY_marketCap,PY_peRatio,PY_interestCoverage,PY_dividendYield,Unnamed: 0,sector,S&P500_return,MarketCap_Tier
0,ZYXI,12/30/2022,13.910000,13.910000,2022,8.964234,ZYXI-2022,0.551722,FY,3.400606,0.446355,0.181356,2.831626e+08,16.556313,235.431579,0.000004,637,Healthcare,-0.1817,small-cap
1,ZYXI,12/31/2021,9.063600,8.964234,2021,12.102167,ZYXI-2021,-0.259287,FY,2.150586,0.243559,0.021956,6.120128e+08,67.446863,539.421053,NaN,637,Healthcare,0.2875,small-cap
2,ZYXI,12/31/2020,12.236400,12.102167,2020,7.076080,ZYXI-2020,0.710293,FY,1.274336,0.266010,0.176639,3.156315e+08,33.252370,2213.200000,0.007167,637,Healthcare,0.1837,small-cap
3,ZYXI,12/31/2019,7.154500,7.076080,2019,2.643415,ZYXI-2019,1.676871,FY,0.892701,0.267164,0.263137,1.251366e+08,13.100560,67.337662,NaN,637,Healthcare,0.3122,micro-cap
4,ZYXI,12/31/2018,2.644294,2.643415,2018,2.783678,ZYXI-2018,-0.050388,FY,0.662452,0.208218,0.233521,1.549919e+08,21.044386,6.168276,NaN,637,Healthcare,-0.0456,micro-cap
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29612,A,12/31/2021,159.650000,158.397827,2021,116.890472,A-2021,0.355096,FY,17.278317,2.326861,2.980583,3.612210e+10,50.239361,10.846154,0.006146,2520,Healthcare,0.2875,large-cap
29613,A,12/31/2020,118.490000,116.890472,2020,83.618065,A-2020,0.397909,FY,16.034161,3.326087,3.170807,2.600794e+10,24.283790,12.716216,0.007921,2520,Healthcare,0.1837,large-cap
29614,A,12/31/2019,85.310000,83.618065,2019,65.562431,A-2019,0.275396,FY,15.073620,0.969325,3.334356,2.358610e+10,74.639555,12.373333,0.008098,2520,Healthcare,0.3122,large-cap
29615,A,12/31/2018,67.460000,65.562431,2018,64.488472,A-2018,0.016654,FY,13.429429,2.054054,2.669670,2.305692e+10,33.708946,10.645570,0.007373,2520,Healthcare,-0.0456,large-cap


## Add binary target: "Did stock beat the S&P500 in the given year?"

In [83]:
# Add binary target column (as Yes/No):

conditions2 = [
    stock_data4['Return'] > stock_data4['S&P500_return'],
    stock_data4['Return'] <= stock_data4['S&P500_return']
]

values2 = ['Yes', 'No']

stock_data4['Beat_S&P?'] = np.select(conditions2, values2)

stock_data4

,symbol,date,close,adjClose,Year,PY_adjClose,Key,Return,period,PY_revenuePerShare,...,PY_operatingCashFlowPerShare,PY_marketCap,PY_peRatio,PY_interestCoverage,PY_dividendYield,Unnamed: 0,sector,S&P500_return,MarketCap_Tier,Beat_S&P?
0,ZYXI,12/30/2022,13.910000,13.910000,2022,8.964234,ZYXI-2022,0.551722,FY,3.400606,...,0.181356,2.831626e+08,16.556313,235.431579,0.000004,637,Healthcare,-0.1817,small-cap,Yes
1,ZYXI,12/31/2021,9.063600,8.964234,2021,12.102167,ZYXI-2021,-0.259287,FY,2.150586,...,0.021956,6.120128e+08,67.446863,539.421053,NaN,637,Healthcare,0.2875,small-cap,No
2,ZYXI,12/31/2020,12.236400,12.102167,2020,7.076080,ZYXI-2020,0.710293,FY,1.274336,...,0.176639,3.156315e+08,33.252370,2213.200000,0.007167,637,Healthcare,0.1837,small-cap,Yes
3,ZYXI,12/31/2019,7.154500,7.076080,2019,2.643415,ZYXI-2019,1.676871,FY,0.892701,...,0.263137,1.251366e+08,13.100560,67.337662,NaN,637,Healthcare,0.3122,micro-cap,Yes
4,ZYXI,12/31/2018,2.644294,2.643415,2018,2.783678,ZYXI-2018,-0.050388,FY,0.662452,...,0.233521,1.549919e+08,21.044386,6.168276,NaN,637,Healthcare,-0.0456,micro-cap,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29612,A,12/31/2021,159.650000,158.397827,2021,116.890472,A-2021,0.355096,FY,17.278317,...,2.980583,3.612210e+10,50.239361,10.846154,0.006146,2520,Healthcare,0.2875,large-cap,Yes
29613,A,12/31/2020,118.490000,116.890472,2020,83.618065,A-2020,0.397909,FY,16.034161,...,3.170807,2.600794e+10,24.283790,12.716216,0.007921,2520,Healthcare,0.1837,large-cap,Yes
29614,A,12/31/2019,85.310000,83.618065,2019,65.562431,A-2019,0.275396,FY,15.073620,...,3.334356,2.358610e+10,74.639555,12.373333,0.008098,2520,Healthcare,0.3122,large-cap,No
29615,A,12/31/2018,67.460000,65.562431,2018,64.488472,A-2018,0.016654,FY,13.429429,...,2.669670,2.305692e+10,33.708946,10.645570,0.007373,2520,Healthcare,-0.0456,large-cap,Yes


In [84]:
   ## re-order columns
clean_df = stock_data4.copy()
clean_df = clean_df.loc[:,['Key', 'symbol', 'period', 'Year', 'Return', 'S&P500_return', 'Beat_S&P?', 'sector', 'MarketCap_Tier',
                           'PY_revenuePerShare','PY_netIncomePerShare', 'PY_operatingCashFlowPerShare', 'PY_marketCap', 'PY_peRatio',
                               	'PY_interestCoverage', 'PY_dividendYield' ,'close', 'adjClose', 'PY_adjClose', 'date']]

clean_df                                   


,Key,symbol,period,Year,Return,S&P500_return,Beat_S&P?,sector,MarketCap_Tier,PY_revenuePerShare,PY_netIncomePerShare,PY_operatingCashFlowPerShare,PY_marketCap,PY_peRatio,PY_interestCoverage,PY_dividendYield,close,adjClose,PY_adjClose,date
0,ZYXI-2022,ZYXI,FY,2022,0.551722,-0.1817,Yes,Healthcare,small-cap,3.400606,0.446355,0.181356,2.831626e+08,16.556313,235.431579,0.000004,13.910000,13.910000,8.964234,12/30/2022
1,ZYXI-2021,ZYXI,FY,2021,-0.259287,0.2875,No,Healthcare,small-cap,2.150586,0.243559,0.021956,6.120128e+08,67.446863,539.421053,NaN,9.063600,8.964234,12.102167,12/31/2021
2,ZYXI-2020,ZYXI,FY,2020,0.710293,0.1837,Yes,Healthcare,small-cap,1.274336,0.266010,0.176639,3.156315e+08,33.252370,2213.200000,0.007167,12.236400,12.102167,7.076080,12/31/2020
3,ZYXI-2019,ZYXI,FY,2019,1.676871,0.3122,Yes,Healthcare,micro-cap,0.892701,0.267164,0.263137,1.251366e+08,13.100560,67.337662,NaN,7.154500,7.076080,2.643415,12/31/2019
4,ZYXI-2018,ZYXI,FY,2018,-0.050388,-0.0456,No,Healthcare,micro-cap,0.662452,0.208218,0.233521,1.549919e+08,21.044386,6.168276,NaN,2.644294,2.643415,2.783678,12/31/2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29612,A-2021,A,FY,2021,0.355096,0.2875,Yes,Healthcare,large-cap,17.278317,2.326861,2.980583,3.612210e+10,50.239361,10.846154,0.006146,159.650000,158.397827,116.890472,12/31/2021
29613,A-2020,A,FY,2020,0.397909,0.1837,Yes,Healthcare,large-cap,16.034161,3.326087,3.170807,2.600794e+10,24.283790,12.716216,0.007921,118.490000,116.890472,83.618065,12/31/2020
29614,A-2019,A,FY,2019,0.275396,0.3122,No,Healthcare,large-cap,15.073620,0.969325,3.334356,2.358610e+10,74.639555,12.373333,0.008098,85.310000,83.618065,65.562431,12/31/2019
29615,A-2018,A,FY,2018,0.016654,-0.0456,Yes,Healthcare,large-cap,13.429429,2.054054,2.669670,2.305692e+10,33.708946,10.645570,0.007373,67.460000,65.562431,64.488472,12/31/2018


In [85]:
# Save the dataframe to CSV file
clean_df.to_csv('Stock_List_v2.csv')
print("Stock listed exported to working directory")

Stock listed exported to working directory
